<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="../figures/PDSH-cover-small.png">
*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) | [Contents](Index.ipynb) | [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) >

# Understanding Data Types in Python

In [1]:
import sys
print(sys.executable)
# 현재 실행 중인 Python 인터프리터의 실제 경로를 확인


c:\venvs\DS_Coding\Scripts\python.exe


### Python 개발에서는 자주 발생하는 문제:

- 여러 버전의 Python 설치

- 가상환경(venv) 여러 개 존재

- Jupyter 커널과 터미널이 서로 다른 Python을 사용

- VSCode에서 어떤 Python이 선택되었는지 헷갈림

**Effective data-driven science and computation requires understanding how data is stored and manipulated.**
This section outlines and contrasts **how arrays of data are handled in the Python language itself, and how NumPy improves on this.**
Understanding this difference is fundamental to understanding much of the material throughout the rest of the book.

**Users of Python are often drawn-in by its ease of use, one piece of which is dynamic typing.**
While a statically-typed language like C or Java requires each variable to be explicitly declared, a dynamically-typed language like Python skips this specification. For example, in C you might specify a particular operation as follows:

```C
/* C 코드 */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

While in Python the equivalent operation could be written this way:

```python
# 파이썬 코드
result = 0
for i in range(100):
    result += i
```

Notice the main difference: in C, the data types of each variable are explicitly declared, while **in Python the types are dynamically inferred.** This means, for example, that we can **assign any kind of data to any variable**:

```python
# 파이썬 코드
x = 4
x = "four"
```

Here we've **switched the contents of ``x`` from an integer to a string.** The same thing in C would lead (depending on compiler settings) to a compilation error or other unintented consequences:

```C
/* C 코드 */
int x = 4;
x = "four";  // 실패
```

This sort of flexibility is one piece that makes Python and other dynamically-typed languages convenient and easy to use.
***<u>Understanding *how* this works is an important piece of learning to analyze data efficiently and effectively with Python.***</u>
But what this type-flexibility also points to is the fact that Python variables are more than just their value; they also contain extra information about the type of the value. We'll explore this more in the sections that follow.

In [ ]:
x = 10        # 이 순간 x → 정수(int) 객체 10을 가리킴
x = "hello"   # 이제 x → 문자열(str) 객체 "hello"를 가리킴
### “파이썬은 변수의 타입이 바뀌는 게 아니라 변수(이름)가 다른 타입의 객체를 가리키게 되는 것”

### 가변(mutable) vs 불변(immutable)

- 파이썬 객체는 크게 두 부류:

1) 불변(immutable): 생성 후 내부 값을 바꿀 수 없는 객체

    int, float, str, tuple, frozenset 등

2) 가변(mutable): 생성 후 내부 값을 바꿀 수 있는 객체

    list, dict, set, 대부분의 사용자 정의 클래스 인스턴스 등

In [3]:
a = 10
b = a
print(id(a), id(b))  # 같은 주소일 가능성이 큼 (같은 객체 가리킴)

a = a + 1
# a = a + 1 을 했을 때 기존 정수 객체 10이 수정되는 것이 아니라
# 새로운 정수 객체 11이 만들어지고, a가 그걸 가리키게 됨
print(a, b)          # a=11, b=10
print(id(a), id(b)) 

140712366994504 140712366994504
11 10
140712366994536 140712366994504


## A Python Integer Is More Than Just an Integer

**The standard Python implementation is written in C.**
This means that every Python object is simply a cleverly-disguised C structure, which contains not only its value, but other information as well. For example, when we define an integer in Python, such as ``x = 10000``, ``x`` is not just a "raw" integer. It's actually a pointer to a compound C structure, which contains several values.
Looking through the Python 3.4 source code, we find that **the integer (long) type definition effectively looks like this (once the C macros are expanded):**

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

A single integer in Python 3.4 actually contains four pieces:

- ``ob_refcnt``, a reference count that helps Python silently handle memory allocation and deallocation
- ``ob_type``, which encodes the type of the variable
- ``ob_size``, which specifies the size of the following data members
- ``ob_digit``, which contains the actual integer value that we expect the Python variable to represent.

This means that **there is some overhead in storing an integer in Python as compared to an integer in a compiled language like C**, as illustrated in the following figure:

![Integer Memory Layout](../figures/cint_vs_pyint.png)

Here **``PyObject_HEAD`` is the part of the structure containing the reference count, type code, and other pieces mentioned before.**

Notice the difference here: a C integer is essentially a label for a position in memory whose bytes encode an integer value.
***<u>A Python integer is a pointer to a position in memory containing all the Python object information, including the bytes that contain the integer value.***</u>
This extra information in the Python integer structure is what allows Python to be coded so freely and dynamically.
All this additional information in Python types comes at a cost, however, which becomes especially apparent in structures that combine many of these objects.

## A Python List Is More Than Just a List

Let's consider now what happens when we use a Python data structure that holds many Python objects.
The standard mutable multi-element container in Python is the list.
We can create a list of integers as follows:

In [1]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
type(L[0])

int

Or, similarly, a list of strings:

In [3]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [4]:
type(L2[0])

str

Because of **Python's dynamic typing, we can even create heterogeneous lists:**

In [5]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

But this flexibility comes at a cost: to allow these flexible types, ***<u>each item in the list must contain its own type info, reference count, and other information–that is, each item is a complete Python object.***</u>
In the special case that all variables are of the same type, much of this information is redundant: it can be much more efficient to store data in a fixed-type array.
***<u>The difference between a dynamic-type list and a fixed-type (NumPy-style) array***</u> is illustrated in the following figure:

![Array Memory Layout](../figures/array_vs_list.png)

At the implementation level, the array essentially contains a single pointer to one contiguous block of data.
***<u>The Python list, on the other hand, contains a pointer to a block of pointers, each of which in turn points to a full Python object like the Python integer***</u> we saw earlier.
Again, the advantage of the list is flexibility: because each list element is a full structure containing both data and type information, the list can be filled with data of any desired type.
***<u>Fixed-type NumPy-style arrays lack this flexibility, but are much more efficient for storing and manipulating data.***</u>

## Fixed-Type Arrays in Python

Python offers several different options for storing data in efficient, fixed-type data buffers.
***<u>The built-in ``array`` module (available since Python 3.3) can be used to create dense arrays of a uniform type:***</u>

파이썬에서는 range 함수가 range 객체를 생성하며, 이는 **이터러블(iterable)**이자 **반복자(iterator)**로 동작할 수 있는 특별한 객체. 
- range 객체는 메모리를 효율적으로 사용하기 위해 숫자의 리스트를 미리 생성하지 않고 필요할 때 계산하여 제공.

In [7]:
num = range(10)
print(num)

range(0, 10)


list() 함수는 인자로 받은 이터러블(iterable) 객체를 **리스트(list)**로 변환하는 역할

- list()를 사용하여, 튜플, 문자열, range 객체, 집합 등 다양한 이터러블을 리스트로 변환.

In [6]:
import array
L = list(range(10))
A = array.array('i', L)
A

array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Here ``'i'`` is a type code indicating the contents are integers.

**<u>Much more useful, however, is the ``ndarray`` object of the NumPy package.**</u>
While **Python's ``array`` object provides efficient storage of array-based data, NumPy adds to this efficient *operations* on that data.**
We will explore these operations in later sections; here we'll demonstrate several ways of creating a NumPy array.

We'll start with the standard NumPy import, under the alias ``np``:

array.array (표준 라이브러리)
- array는 Python 표준 라이브러리
- 	1차원 배열만 지원
- 단순 저장/기초 연산만
- 리스트보다 빠름 (단순함 때문)
- 메모리 절약, 구조화된 배열

numpy.ndarray (NumPy 라이브러리)
- ndarray는 NumPy에서 제공
- 다차원 배열(1D, 2D, 3D...) 지원
- 벡터 연산, 행렬 연산, 브로드캐스팅 등 수학적 기능 풍부
- C로 구현돼 훨씬 빠름, 과학계산 최적화
- 수학, 과학, 머신러닝 등 고성능 연산

In [6]:
%pip install numpy

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/aa/44/9fe81ae1dcc29c531843852e2874080dc441338574ccc4306b39e2ff6e59/numpy-2.3.5-cp311-cp311-win_amd64.whl.metadata
  Using cached numpy-2.3.5-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.5-cp311-cp311-win_amd64.whl (13.1 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np

## Creating Arrays from Python Lists

First, we can use ``np.array`` to create arrays from Python lists:

In [3]:

# 정수 배열:
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

Remember that **unlike Python lists, NumPy is constrained to arrays that all contain the same type**.
If types do not match, NumPy will upcast if possible (here, integers are up-cast to floating point):

In [4]:
np.array([3.14, 4, 2, 3])

array([3.14, 4.  , 2.  , 3.  ])

If we want to **explicitly set the data type of the resulting array, we can use the ``dtype`` keyword:**

In [5]:
np.array([1, 2, 3, 4], dtype='float32')

array([1., 2., 3., 4.], dtype=float32)

Finally, unlike Python lists, NumPy arrays can explicitly be multi-dimensional; here's one way of initializing a multidimensional array using a list of lists:

배열 기반 데이터의 주요 특징
- 고정 데이터 타입: 배열의 모든 요소가 같은 데이터 타입
>  array('i', [...])은 정수형(int) 배열을 뜻하며, 배열에 포함된 모든 요소가 int
> 메모리가 고정된 크기로 할당되므로 효율적으로 관리

- 연속적 메모리 할당: 배열은 메모리에 연속적으로 배치되어 있어, 인덱스를 이용한 접근 속도가 매우 빠름
> 리스트와 달리 각 요소가 고정된 크기를 가지므로, 특정 인덱스로의 접근은 리스트보다 효율적

- 메모리 사용 최적화: 리스트는 각 요소에 대한 추가 정보와 참조를 저장하기 위해 오버헤드가 존재
> 배열은 고정된 크기의 데이터로 구성되어 있기 때문에 더 적은 메모리를 사용

- 파이썬에서는 기본적으로 array 모듈을 사용해 배열을 생성
> 다차원 배열이나 복잡한 계산이 필요할 경우 numpy의 ndarray와 같은 라이브러리를 사용

In [5]:
# 리스트를 중첩시키면 다차원 배열이 됨
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

The inner lists are treated as rows of the resulting two-dimensional array.

파이썬의 기본 array 모듈의 array 객체와 numpy 라이브러리의 ndarray 객체는 모두 배열 기반의 데이터를 다루지만, 몇 가지 중요한 차이점.
 - array는 단순한 1차원 배열에 특화되어 있는 반면, ndarray는 다차원 배열 연산을 포함해 보다 복잡한 수치 계산에 최적화.

주요 차이점::
1. 다차원 지원

- array: 기본 array 모듈의 배열은 1차원 배열만을 지원. 다차원 배열이 필요한 경우 별도의 방법을 사용해야 하며, 기본 array만으로는 다차원 데이터를 효율적으로 처리하기 어렵다.
- ndarray: numpy의 ndarray는 다차원 배열을 기본으로 지원하며, 복잡한 다차원 배열 연산과 인덱싱이 가능합니다.

2. 데이터 타입

- array: 하나의 고정 데이터 타입을 사용하며, 정수('i'), 부동소수점('f') 등의 기본 타입을 지정해야 한다. 이 때문에 다차원 배열 또는 복잡한 데이터 타입을 지원하기에 적합하지 않다.

- ndarray: 훨씬 다양한 데이터 타입을 지원하며, 정수, 부동소수점, 복소수, 불리언 등 다양한 타입으로 배열을 생성할 수 있다. 커스텀 데이터 타입을 만들 수도 있어 과학 계산에 유용.

3. 벡터화 및 고속 연산

- array: 기본 array는 단순히 저장을 목적으로 한 배열 객체이며, 고속 연산을 위한 최적화가 되어 있지 않다. 반복문을 사용해 연산을 처리해야 하는 경우가 많다.
- ndarray: numpy는 **벡터화(vectorization)**와 **브로드캐스팅(broadcasting)**이라는 개념을 통해 배열 연산을 매우 빠르게 수행. 전체 배열에 대해 반복문 없이 한 번에 연산을 적용할 수 있어 성능이 훨씬 뛰어나다.

4. 다양한 연산 함수 지원

- array: 기본적인 인덱싱과 슬라이싱, 요소 추가/삭제 등의 간단한 기능만 지원.
- ndarray: numpy는 수학적 연산, 통계, 선형 대수, 푸리에 변환 등 다양한 함수들을 내장하여 ndarray 객체에서 직접 사용할 수 있다. 예를 들어, 행렬 곱, 역행렬, 트랜스포즈, 통계 값 계산 등을 간단히 수행할 수 있다.

5. 메모리 및 성능 효율성

- array: 1차원 배열에 대해 효율적이지만, 고성능 대규모 연산에 특화되어 있지는 않다.
- ndarray: 대용량 데이터를 효율적으로 처리하며, numpy는 C로 구현되어 있어 일반 파이썬 반복문보다 훨씬 빠른 성능을 제공합.

## Creating Arrays from Scratch

Especially for larger arrays, it is more efficient to create arrays from scratch using routines built into NumPy.
Here are several examples:

In [6]:
# 0으로 채원진 길이 10의 정수 배열 만들기
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
# # 1로 채워진 3x5 부동소수 배열 만들기
np.ones((3, 5), dtype=float)

array([[ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]])

In [14]:
# 3.14로 채워진 배열 만들기
np.full((3, 5), 3.14)

array([[ 3.14,  3.14,  3.14,  3.14,  3.14],
       [ 3.14,  3.14,  3.14,  3.14,  3.14],
       [ 3.14,  3.14,  3.14,  3.14,  3.14]])

In [ ]:
# 선형 수열로 채워진 배열 만들기
# 0에서 시작하여 2씩 더하여 20까지 채움
# (내장 함수인 range() 와 유사함)
np.arange(0, 20, 2) ##기억해야 한다 

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [ ]:
# 0과 1 사이에 일정한 간격을 가진 다섯 개의 값으로 채원진 배열 만들기
np.linspace(0, 1, 5) ## 기억해야 한다 

array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ])

In [ ]:
# 균등하게 분포된 3x3 배열 만들기
# 0과 1 사이의 난수로 채워짐
np.random.random((3, 3)) ## 기억해야 한다.

array([[ 0.99844933,  0.52183819,  0.22421193],
       [ 0.08007488,  0.45429293,  0.20941444],
       [ 0.14360941,  0.96910973,  0.946117  ]])

In [18]:
# 정규 분포(평균=0, 표준편차=1)의 난수로 채워진 3x3 배열 만들기
np.random.normal(0, 1, (3, 3))

array([[ 1.51772646,  0.39614948, -0.10634696],
       [ 0.25671348,  0.00732722,  0.37783601],
       [ 0.68446945,  0.15926039, -0.70744073]])

In [19]:
# 구간 [0,10] 사이의 임의의 정수로 채워진 3x3 배열 만들기
np.random.randint(0, 10, (3, 3))

array([[2, 3, 4],
       [5, 7, 8],
       [0, 5, 0]])

In [20]:
# 3x3 단위 행렬 만들기
np.eye(3)

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [13]:
# 세 개의 정수를 가지는 초기화되지 않은 배열 만들기
# 값은 해당 메모리 위치에 이미 존재하고 있는 값으로 채워짐
np.empty(3)

array([8.86246056e+281, 1.50755939e+161, 1.07349387e+272])

## NumPy Standard Data Types

**NumPy arrays contain values of a single type, so it is important to have detailed knowledge of those types and their limitations.**
Because NumPy is built in C, the types will be familiar to users of C, Fortran, and other related languages.

The standard NumPy data types are listed in the following table.
Note that when constructing an array, they can be specified using a string:

```python
np.zeros(10, dtype='int16')
```

Or using the associated NumPy object:

```python
np.zeros(10, dtype=np.int16)
```

| Data type	    | Description |
|---------------|-------------|
| ``bool_``     | Boolean (True or False) stored as a byte |
| ``int_``      | Default integer type (same as C ``long``; normally either ``int64`` or ``int32``)| 
| ``intc``      | Identical to C ``int`` (normally ``int32`` or ``int64``)| 
| ``intp``      | Integer used for indexing (same as C ``ssize_t``; normally either ``int32`` or ``int64``)| 
| ``int8``      | Byte (-128 to 127)| 
| ``int16``     | Integer (-32768 to 32767)|
| ``int32``     | Integer (-2147483648 to 2147483647)|
| ``int64``     | Integer (-9223372036854775808 to 9223372036854775807)| 
| ``uint8``     | Unsigned integer (0 to 255)| 
| ``uint16``    | Unsigned integer (0 to 65535)| 
| ``uint32``    | Unsigned integer (0 to 4294967295)| 
| ``uint64``    | Unsigned integer (0 to 18446744073709551615)| 
| ``float_``    | Shorthand for ``float64``.| 
| ``float16``   | Half precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
| ``float32``   | Single precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
| ``float64``   | Double precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
| ``complex_``  | Shorthand for ``complex128``.| 
| ``complex64`` | Complex number, represented by two 32-bit floats| 
| ``complex128``| Complex number, represented by two 64-bit floats| 

More advanced type specification is possible, such as specifying big or little endian numbers; for more information, refer to the [NumPy documentation](http://numpy.org/).
NumPy also supports compound data types, which will be covered in [Structured Data: NumPy's Structured Arrays](02.09-Structured-Data-NumPy.ipynb).

<!--NAVIGATION-->
< [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) | [Contents](Index.ipynb) | [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) >